In [ ]:
# Hello there.   This is a do-over of the floating preferred analyzer

# Time to develop clean modules. Let's start with reading in a list of prefs
# The input data consists of three separate tables:
# a) A listing of tickers and their configurations
# b) A list of ratings for each company (equivalently, of the ticker)
# c) The current interest rate environment, which has only the 3-month rate and the prime rate



In [ ]:
# Imports go here
import pandas as pd
import requests_cache
# local ones
import prefcode as pc


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Set up cache
session_cached = requests_cache.CachedSession('yfinance.cache',expire_after=3600)
session_uncached = requests_cache.CachedSession('yfinance2.cache',expire_after=30)


In [ ]:
# Read the prefs database

print("Setting up databases")
df, interest_db = pc.setup_databases()

print("Fetching price data")
df = pc.fetch_prices(df, session_cached, fetch=True)

print("Updating dividends and current yield")
df = pc.update_div_and_yield(df, interest_db,price_column="Price")

print("Updating market spread for later calculation")
df2 = pc.update_market_spread(df, interest_db)

print("Dropping prime-related preferreds. Maybe another day")
tdf = df[df['Type'] == 'T'].copy()

print("Calculating scenarios")
scenarios = {"010":  [0.10,  0.05],
             "018":  [0.18,  0.25],
             "030":  [0.30,  0.50],
             "050":  [0.50,  0.20]}

pc.update_expected_yield(tdf, scenarios)


In [ ]:
print("Here are the highest yield items")
tdf.sort_values(by='ExpYield', ascending=False).head(15)

In [ ]:
print("And uniquified by issuer")
# Uniquify by parent
tdf['Parent'] = [x.split('.')[0] for x in tdf['Ticker']]
#tdf
uniq_parent = pc.summarize_best_by_column(tdf,'ExpYield','Parent')

uniq_parent = uniq_parent.reindex(columns=["Ticker", "Rating", 
                                           "Spread","CurYieldPct","ExpYield"])
# make it a bit more readable

uniq_parent['CurYield'] = [round(x,3) for x in uniq_parent['CurYieldPct']]
uniq_parent['ExpYield'] = [round(x,3) for x in uniq_parent['ExpYield']]
uniq_parent.drop(columns=['CurYieldPct'], errors='ignore', inplace=True)
uniq_parent.sort_values(by='ExpYield', ascending=False).head(15)


In [ ]:
best_scn_df = pc.summarize_best_by_column(tdf, 'ExpYield')
best_scn_df = best_scn_df.reindex(columns=["Ticker", "Rating", "Spread","ExpYield", "MSpread"])

print("And here are the best by rating level")
best_scn_df.sort_values(by=['Rating'])

In [ ]:
# Now, check market spread and flag if below average for the group
# That could result in a re-rating
mdf = pc.calculate_avg_per_rating(tdf, column='MSpread')
mdf